In [76]:
import pandas as pd
import kaggle
import zipfile
import regex as re
from utils import Data_Cleaner, BytePairEncoder
import tqdm
import pickle

In [ ]:
# Download the dataset using kaggle API
!kaggle datasets download -d disisbig/hindi-text-short-summarization-corpus


In [59]:
# Read the CSV file
df = pd.read_csv('data/train.csv')

# Display the first few rows
df.head()

,headline,article
0,EXCLUSIVE: दिल्ली में डीजल टैक्सियों पर बैन से...,दिल्ली में सुप्रीम कोर्ट के डीज़ल टैक्सियों को...
1,जॉर्डन: राष्ट्रपति मुखर्जी ने 86 करोड़ डॉलर के...,जॉर्डन के ऐतिहासिक दौरे पर पहुंचे राष्ट्रपति प...
2,UN में पाकिस्तान की राजदूत मलीहा लोधी ने कराई ...,पाकिस्तानी नेताओं को विवादित और हास्यास्पद बया...
3,38 देशों में पीएम नरेंद्र मोदी बायोपिक को रिली...,पीएम नरेंद्र मोदी बायोपिक में विवेक ओबेरॉय ने ...
4,13 अगस्त 2011: दिनभर की बड़ी खबरें पढ़ें,"देश, दुनिया, महानगर, खेल, आर्थिक और बॉलीवुड मे..."


In [60]:
len(df['article'])

266607

In [61]:
articles_text_corpus = ' '.join(df['article'][:1000].values)

# Clean the corpus by removing extra whitespace
articles_text_corpus = ' '.join(articles_text_corpus.split())

# Print the first 500 characters to verify
print("First 500 characters of the corpus:")
print(articles_text_corpus[:500])

# Print some basic statistics
print(f"\nTotal number of characters in corpus: {len(articles_text_corpus)}")
print(f"Total number of words in corpus: {len(articles_text_corpus.split())}")
print(f"Total number of unique words in corpus: {len(set(articles_text_corpus.split()))}")

First 500 characters of the corpus:
दिल्ली में सुप्रीम कोर्ट के डीज़ल टैक्सियों को बंद करने के फैसले के बाद हजारों टैक्सी ड्राइवरों की रोजी रोटी पर तो असर पड़ा ही है, लेकिन अब दिल्ली पर एक और नई मुसीबत आ गई है. चुनाव आयोग राजधानी के 13 वार्ड में उपचुनाव करवा रहा है, लेकिन चुनावों से दो हफ्ते पहले चुनाव आयोग में कामकाज ठप्प हो गया है. कमीशन ने किराए पर ली थी डीजल गाड़ियां दरअसल कमीशन ने लगभग सौ गाड़ियां चुनाव के कामकाज को करने के लिए किराए पर लीं, जिनमें सभी डीज़ल से चलने वाली टैक्सी थी. इन्हीं टैक्सियों से चुनाव अधिकारी से लेकर चु

Total number of characters in corpus: 1861278
Total number of words in corpus: 366465
Total number of unique words in corpus: 33830


In [62]:
def Data_Cleaner(text):
    import regex as re
    # Define patterns
    pat_remove_English_words = re.compile(r""" ?[a-zA-Z]+| ?\p{N}+""")
    pat_non_Devnagari_words = re.compile(r""" ?[^\u0900-\u097F\s]""")
    pat_keep_Devnagari_words = re.compile(r"""( ?[\u0900-\u097F]+)""")

    # Remove special characters and extra spaces
    text = re.sub(pat_remove_English_words, '', text)
    text = re.sub(pat_non_Devnagari_words, '', text)
    text = re.sub(pat_keep_Devnagari_words, r" \1", text)
    text = re.sub(r"\s+", " ", text).strip()
    text = ' '.join(text.split())
    return text

articles_text_corpus = Data_Cleaner(articles_text_corpus)

In [63]:
print(f"\nTotal number of characters in corpus: {len(articles_text_corpus)}")
print(f"Total number of words in corpus: {len(articles_text_corpus.split())}")
print(f"Total number of unique words in corpus: {len(set(articles_text_corpus.split()))}")

print(articles_text_corpus[:1000])



Total number of characters in corpus: 1717664
Total number of words in corpus: 344533
Total number of unique words in corpus: 22704
दिल्ली में सुप्रीम कोर्ट के डीज़ल टैक्सियों को बंद करने के फैसले के बाद हजारों टैक्सी ड्राइवरों की रोजी रोटी पर तो असर पड़ा ही है लेकिन अब दिल्ली पर एक और नई मुसीबत आ गई है चुनाव आयोग राजधानी के वार्ड में उपचुनाव करवा रहा है लेकिन चुनावों से दो हफ्ते पहले चुनाव आयोग में कामकाज ठप्प हो गया है कमीशन ने किराए पर ली थी डीजल गाड़ियां दरअसल कमीशन ने लगभग सौ गाड़ियां चुनाव के कामकाज को करने के लिए किराए पर लीं जिनमें सभी डीज़ल से चलने वाली टैक्सी थी इन्हीं टैक्सियों से चुनाव अधिकारी से लेकर चुनावों का जिम्मा संभालने वाले बाकी कर्मचारी भी एक जगह से दूसरी जगह आते जाते थे अचानक चुनावों से ठीक पहले आई इस परेशानी ने दिल्ली चुनाव आयोग का कामकाज ही ठप्प कर दिया है रियायत के लिए की जा सकती है मांग दिल्ली के राज्य चुनाव अधिकारी राकेश मेहता ने इस मुश्किल का रास्ता निकालने के लिए मंगलवार को दिल्ली के पुलिस कमिश्नर और ट्रांसपोर्ट कमिश्नर की बैठक बुलाई है इस बैठक में राज्य च

In [64]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(articles_text_corpus)))
initial_vocab_size = len(chars)
print(''.join(chars))
print(initial_vocab_size)

 ँंःअआइईउऊऋऌएऐऑओऔकखगघचछजझञटठडढणतथदधनऩपफबभमयरऱलवशषसह़ािीुूृॅेैॉोौ्ॐक़ख़ज़ड़ढ़फ़य़।
74


In [65]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}

In [66]:
tokens = [stoi[c] for c in articles_text_corpus] # convert to a list of integers using custom encoder
print(len(tokens))

1717664


In [67]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
  newids = []
  i = 0
  while i < len(ids):
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  return newids

In [68]:
vocab_size = 4500 # the desired final vocabulary size
num_merges = vocab_size - initial_vocab_size
ids = list(tokens) # copy so we don't destroy the original list

In [69]:
merges = {} # (int, int) -> int
for i in tqdm.tqdm(range(num_merges)):
  stats = get_stats(ids)
  pair = max(stats, key=stats.get)
  try:
    pair_char = "".join([itos[pair[0]], itos[pair[1]]])
  except KeyError:
    print(f"pair {pair} not in vocab")
    break
  stoi[pair_char] = initial_vocab_size + i
  itos[initial_vocab_size + i] = pair_char
  idx = initial_vocab_size + i
  # print(f"merging {pair} into a new token {idx}")
  ids = merge(ids, pair, idx)
  merges[pair] = idx

100%|██████████| 4426/4426 [31:27<00:00,  2.34it/s]   


In [70]:

print("tokens length:", len(tokens))
print("ids length:", len(ids))
print(f"compression ratio: {len(tokens) / len(ids):.2f}X")

tokens length: 1717664
ids length: 444729
compression ratio: 3.86X


In [73]:
def encode(text):
  # given a string, return list of integers (the tokens)
  tokens = [stoi[c] for c in text]
  while len(tokens) >= 2:
    stats = get_stats(tokens)
    pair = min(stats, key=lambda p: merges.get(p, float("inf")))
    if pair not in merges:
      break # nothing else can be merged
    idx = merges[pair]
    tokens = merge(tokens, pair, idx)
  return tokens

def decode(ids):
  text = "".join([itos[idx] for idx in ids])
  return text

print(encode("ये दिल्ली है मेरे यार"))
print(decode(encode("ये दिल्ली है मेरे यार")))

[269, 519, 92, 1329, 42, 85]
ये दिल्ली है मेरे यार


In [ ]:
for idx in [269, 519, 92, 1329, 42, 85]:
    print(idx,">>",itos[idx])

In [72]:
print(len(stoi))
print(len(itos))

4500
4500


In [77]:
# Save the mappings
with open('stoi.pkl', 'wb') as f:
    pickle.dump(stoi, f)

with open('itos.pkl', 'wb') as f:
    pickle.dump(itos, f) 

In [78]:
with open('merges.pkl', 'wb') as f:
    pickle.dump(merges, f) 